In [1]:
import numpy as np
import pandas as pd

In [2]:
df_train_X = pd.read_csv('../data/beer_features_train.csv', index_col=0)
df_train_y = pd.read_csv('../data/beer_labels_panel_train.csv', index_col=0)
df_test_X = pd.read_csv('../data/beer_features_test.csv', index_col=0)
df_test_y = pd.read_csv('../data/beer_labels_panel_test.csv', index_col=0)

df_train_X.shape, df_train_y.shape, df_test_X.shape, df_test_y.shape

((175, 231), (175, 50), (75, 231), (75, 50))

In [3]:
train_X = df_train_X.values
train_y = df_train_y.values
test_X = df_test_X.values
test_y = df_test_y.values

In [4]:
train_X.min()
train_y.min()
test_X.min()
test_y.min()

-2.580113937141425

In [26]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from hyperdt.ensemble import HyperbolicRandomForestClassifier as HyperRF
from sklearn.ensemble import RandomForestRegressor as SklearnRF

models = []

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(train_X, train_y)):
    # regressor = xgb.XGBRegressor(tree_method='hist', seed=42)
    regressor = SklearnRF(random_state=42)
    # regressor = HyperRF(random_state=42)
    # for tree in regressor.trees:
    #     tree.skip_hyperboloid_check = True

    fold_train_X, fold_val_X = train_X[train_idx], train_X[val_idx]
    fold_train_y, fold_val_y = train_y[train_idx], train_y[val_idx]

    # Fitting the model
    regressor.fit(fold_train_X, fold_train_y)

    # Predict the model
    pred = regressor.predict(fold_val_X)

    mae = MAE(fold_val_y, pred)
    r2 = r2_score(fold_val_y, pred)
    # ap = AP(fold_val_y, pred)


    print("Mean MAE : % f" % mae)
    print("Mean R-squared : % f" % r2)
    # print("Mean Average Precision : % f" % ap)
    models.append(regressor)

Mean MAE :  0.652151
Mean R-squared :  0.126738
Mean MAE :  0.647558
Mean R-squared :  0.209197
Mean MAE :  0.594252
Mean R-squared :  0.198723
Mean MAE :  0.654464
Mean R-squared :  0.162244
Mean MAE :  0.584877
Mean R-squared :  0.190215


In [22]:
preds = []
for model in models:
    pred = model.predict(test_X)
    print(pred.shape)
    preds.append(pred)

preds = np.array(preds)

(75, 50)
(75, 50)
(75, 50)
(75, 50)
(75, 50)


In [23]:
for pred in [np.mean(preds, axis=0)]:
    mae = MAE(test_y, pred)
    r2 = r2_score(test_y, pred)
    # ap = AP(fold_val_y, pred)


    print("Mean MAE : % f" % mae)
    print("Mean R-squared : % f" % r2)
    # print("Mean Average Precision : % f" % ap)

Mean MAE :  0.631028
Mean R-squared :  0.245145


XGBregressor:
Mean MAE :  0.638011
Mean R-squared :  0.206903

SklearnRFregressor
Mean MAE :  0.631028
Mean R-squared :  0.245145